In [1]:
import numpy as np
import pickle
from learnable_crf import LearnableCrf
from lib import *
from scipy.special import expit as sigmoid

In [2]:
with open('cache/df_val_test.pickle', mode='rb') as h:
    df_val, df_test = pickle.load(h)
Y_val = df_val['label']
Y_test = df_test['label']
with open('cache/hex.pickle', mode='rb') as h:
    hex_data = pickle.load(h)
id_fh = map(lambda x: list(max(x, key=len)), hex_data['id_hierarchical_labels'])  # convert to list for addressing
state_space = hex_data['state_space']
# state_space = filter(lambda x: x[:20].any(), state_space)

In [3]:
r = 50
with open('cache/df_train.{}.pickle'.format(r), mode='rb') as h:
    df = pickle.load(h)
leaves = np.nonzero([x[0] in x[1] for x in zip(df['label'], df['pseudo_label'])])[0]
Y_train = df['label'][leaves]

In [4]:
model_Phi_train = np.load('results/caffe_independent/iter_Phi_train.{}.npy'.format(r))[:, leaves]
model_Phi_val = np.load('results/caffe_independent/iter_Phi_val.{}.npy'.format(r))
model_Phi_test = np.load('results/caffe_independent/iter_Phi_test.{}.npy'.format(r))
M = len(model_Phi_train)

In [29]:
def confusion_matrix(Y_predict, Y_truth):
    if Y_predict.dtype == bool:
        cm = np.zeros((27, 27), dtype=int)
        count = np.zeros(27, dtype=int)
        for i, y in enumerate(Y_predict):
            count[id_fh[Y_truth[i]]] += 1
            cm[id_fh[Y_truth[i]], :] += y
    else:
        cm = np.zeros((20, 27), dtype=int)
        count = np.zeros(20, dtype=int)
        for i, y in enumerate(Y_predict):
            count[Y_truth[i]] += 1
            cm[Y_truth[i], y.argmax()] += 1
    return cm.astype(float) / count[:, None]

In [13]:
def raw_crf(Phi):
    def step(phi):
        scores = map(lambda s: phi[s].sum(), state_space)
#         return np.vstack(tuple(state_space[np.argsort(scores)[i]] for i in range(-5, 0)))
        return state_space[np.argmax(scores)]
    return np.array(map(step, Phi), dtype=bool)

In [14]:
# [get_accuracy(model_Phi_val[i], Y_val, lim_states=False) for i in range(0, M)]
[get_accuracy(raw_crf(model_Phi_val[i]), Y_val) for i in range(0, M)]

[0.25712589073634207,
 0.32660332541567694,
 0.34323040380047504,
 0.33966745843230406,
 0.34026128266033256,
 0.3414489311163896,
 0.3414489311163896,
 0.3414489311163896,
 0.3414489311163896,
 0.3414489311163896]

In [15]:
# [get_accuracy(model_Phi_test[i], Y_test, lim_states=False) for i in range(0, M)]
[get_accuracy(raw_crf(model_Phi_test[i]), Y_test) for i in range(0, M)]

[0.24168646080760095,
 0.3022565320665083,
 0.3182897862232779,
 0.3171021377672209,
 0.3141330166270784,
 0.3165083135391924,
 0.3165083135391924,
 0.3165083135391924,
 0.3165083135391924,
 0.3165083135391924]

In [30]:
np.diagonal(confusion_matrix(raw_crf(model_Phi_test[2]), Y_test))

array([ 0.35714286,  0.06666667,  0.06896552,  0.16666667,  0.03571429,
        0.45      ,  0.5       ,  0.5       ,  0.3125    ,  0.33333333,
        0.375     ,  0.35862069,  0.17808219,  0.03846154,  0.13043478,
        0.14705882,  0.38636364,  0.22758621,  0.40566038,  0.43283582,
        0.78599222,  0.67295597,  0.95341615,  0.92018779,  0.88250319,
        0.54193548,  0.83372365])

In [ ]:
model_Phi_train = sigmoid(model_Phi_train)
model_Phi_val = sigmoid(model_Phi_val)
model_Phi_test = sigmoid(model_Phi_test)
lcrf = [LearnableCrf(model_Phi_train[i], Y_train) for i in range(0, M)]
[get_accuracy(lcrf[i].predict(model_Phi_val[i]), Y_val) for i in range(0, M)]

In [ ]:
[get_accuracy(lcrf[i].predict(model_Phi_test[i]), Y_test) for i in range(0, M)]